<a href="https://colab.research.google.com/github/DNageshNaik/optical-character-recognition-and-translation/blob/main/Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *************   Model 1  *************************

In [2]:
#connecting to the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

KeyboardInterrupt: ignored

In [ ]:
#importing libraries for layers and model
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as tf_keras_backend

tf_keras_backend.set_image_data_format('channels_last')
tf_keras_backend.image_data_format()

'channels_last'

In [ ]:
#describing the model inputs and outputs
input_data = layers.Input(name='the_input', shape=(128,64,1), dtype='float32')  # (None, 128, 64, 1)

# Convolution layer (VGG)
iam_layers = layers.Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.MaxPooling2D(pool_size=(2, 2), name='max1')(iam_layers)  # (None,64, 32, 64)

iam_layers = layers.Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.MaxPooling2D(pool_size=(2, 2), name='max2')(iam_layers)

iam_layers = layers.Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.MaxPooling2D(pool_size=(1, 2), name='max3')(iam_layers)  # (None, 32, 8, 256)

iam_layers = layers.Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.Conv2D(512, (3, 3), padding='same', name='conv6')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.MaxPooling2D(pool_size=(1, 2), name='max4')(iam_layers)

iam_layers = layers.Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)

# CNN to RNN
iam_layers = layers.Reshape(target_shape=((32, 2048)), name='reshape')(iam_layers)
iam_layers = layers.Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(iam_layers)

# RNN layer
# layer ten
iam_layers = layers.Bidirectional(layers.LSTM(units=256, return_sequences=True))(iam_layers)
# layer nine
iam_layers = layers.Bidirectional(layers.LSTM(units=256, return_sequences=True))(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)

# transforms RNN output to character activations:
iam_layers = layers.Dense(80, kernel_initializer='he_normal', name='dense2')(iam_layers)
iam_outputs = layers.Activation('softmax', name='softmax')(iam_layers)

labels = layers.Input(name='the_labels', shape=[16], dtype='float32')
input_length = layers.Input(name='input_length', shape=[1], dtype='int64')
label_length = layers.Input(name='label_length', shape=[1], dtype='int64')

iam_model_pred = None
iam_model_pred = Model(inputs=input_data, outputs=iam_outputs)
iam_model_pred.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, 128, 64, 1)]      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 128, 64, 64)       640       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 64, 64)       256       
_________________________________________________________________
activation (Activation)      (None, 128, 64, 64)       0         
_________________________________________________________________
max1 (MaxPooling2D)          (None, 64, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 64, 32, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 32, 128)      

In [ ]:
#function to return file path and label
def get_paths_and_gts(partition_split_file):
    """
    read a string like(which can be found in each line of words.txt file):
    'a01-000u-00-00 ok 154 408 768 27 51 AT A'
    and extract 'a01-000u-00-00': location of the image with sub-folders, to read it from the directories
                'ok'            : processing status. ok means good, presumably.
                'A'             : ground truth text

    Then, pre-process using function defined above
    """
    
    data_location = '/content/drive/My Drive/AI in Manufac/project/data/words_zips'

    # a list to store paths to images and ground truth texts
    paths_and_gts = []
    
    # open the file
    with open(partition_split_file) as f:
        # go through each line
        for line in f:
            # if a line is empty or commented with #, ignore that line
            if not line or line.startswith('#'):
                continue
            
            # in the text file, each line is seperated with '\n', so `strip` first
            # then string like 'a01-000u-00-00 ok 154 408 768 27 51 AT A' has to split to a list by spaces
            line_split = line.strip().split(' ')
            
            # the first item of the list contains path information, so split that by '-'
            directory_split = line_split[0].split('-')
            
            # now use all the above and concatenate to a string to make a path to an image
            image_location = f'{data_location}/{directory_split[0]}/{directory_split[0]}-{directory_split[1]}/{line_split[0]}.png'
            
            # in a string like 'a01-000u-00-00 ok 154 408 768 27 51 AT A', text from 9th split is the ground truth text.
            gt_text = ' '.join(line_split[8:])
            
            # ignore a sample(image and ground truth text), if the ground truth has more than 16 letters
            # if len(gt_text) > 16:
                # continue
            
            # now, append the image location and ground truth text of that image as a list to 
            paths_and_gts.append([image_location, gt_text])
    
    return paths_and_gts

In [ ]:
# the above function output will look like this
test_files = get_paths_and_gts('/content/drive/My Drive/AI in Manufac/project/data/test_files.txt')
test_files

[['/content/drive/My Drive/AI in Manufac/project/data/words_zips/a01/a01-000u/a01-000u-00-04.png',
  'Mr.']]

In [ ]:
test_files = [['/content/drive/My Drive/AI in Manufac/project/Group2.jpeg',
  'EDUCATION']]
test_files

[['/content/drive/My Drive/AI in Manufac/project/Group2.jpeg', 'EDUCATION']]

In [ ]:
#pre processing
#function to perform padding
def add_padding(img, old_w, old_h, new_w, new_h):
    h1, h2 = int((new_h - old_h) / 2), int((new_h - old_h) / 2) + old_h
    w1, w2 = int((new_w - old_w) / 2), int((new_w - old_w) / 2) + old_w
    img_pad = np.ones([new_h, new_w, 3]) * 255
    img_pad[h1:h2, w1:w2, :] = img
    return img_pad

#function to convert the image to size of (64, 128, 3)
def fix_size(img, target_w, target_h):
    
    h, w = img.shape[:2]
    if w < target_w and h < target_h:
        img = add_padding(img, w, h, target_w, target_h)
    elif w >= target_w and h < target_h:
        new_w = target_w
        new_h = int(h * new_w / w)
        new_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    elif w < target_w and h >= target_h:
        new_h = target_h
        new_w = int(w * new_h / h)
        new_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    else:
        """w>=target_w and h>=target_h """
        ratio = max(w / target_w, h / target_h)
        new_w = max(min(target_w, int(w / ratio)), 1)
        new_h = max(min(target_h, int(h / ratio)), 1)
        new_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    return img

#performs shaping, normalization and coloring
def preprocess(path, img_w, img_h):
    """ Pre-processing image for predicting """
    #print(path)
    img = cv2.imread(path)
    #print(img.shape)
    #resizing the image to particular size (64, 128, 3)
    img = fix_size(img, img_w, img_h)
    #print(img.shape)
    
    #assigining values less than zero to zer0 and greater than zero to 1
    img = np.clip(img, 0, 255)

    #changing the interger to more useful and complex integer
    img = np.uint8(img)

    #convert an image to one color space to another
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #chaging the values datatype to float
    img = img.astype(np.float32)

    #normalization
    img /= 255
    return img

In [ ]:
test_files = [['/content/drive/My Drive/AI in Manufac/project/Group 2.png', 'ME6324']]

In [ ]:
# the above function output will look like this
test_files = get_paths_and_gts('/content/drive/My Drive/AI in Manufac/project/data/test_files.txt')
test_files

[['/content/drive/My Drive/AI in Manufac/project/data/words_zips/a01/a01-000u/a01-000u-00-04.png',
  'Mr.']]

In [ ]:
test_files = [['/content/drive/My Drive/AI in Manufac/project/images/WhatsApp Image 2020-12-10 at 13.14.24 (1).jpeg','']]

In [ ]:
#plot the transformed image
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import numpy as np
sns.reset_orig()
i = 1
for i in range(1):
    #print(test_files[i][1])
    img = cv2.imread(test_files[i][0])
    temp_processed_image = preprocess(path = test_files[i][0], img_w=128, img_h=64)
    fig = plt.figure()
    ax1 = fig.add_subplot(2,2,1)
    ax1.imshow(img)
    ax2 = fig.add_subplot(2,2,2)
    ax2.imshow(temp_processed_image.T)
    i = i+1

AttributeError: ignored

In [ ]:
#will be doing the same thing for the test files and appending both preproccessed image and label to 2 strings
import cv2
import numpy as np
test_images_processed = []
original_test_texts = []
for _, (test_image_path, original_test_text) in enumerate(test_files):
        temp_processed_image = preprocess(path=test_image_path, img_w=128, img_h=64)
        test_images_processed.append(temp_processed_image.T) 
        original_test_texts.append(original_test_text)
     

In [ ]:
#reshaping the test data
test_images_processed = np.array(test_images_processed)
test_images_processed = test_images_processed.reshape(1, 128, 64, 1)
test_images_processed.shape

(1, 128, 64, 1)

In [ ]:
#importing the trained model
iam_model_pred.load_weights(filepath='/content/drive/My Drive/AI in Manufac/project/lstm-model-after-3rd-session.h5')
test_predictions_encoded = iam_model_pred.predict(x=test_images_processed)

# use CTC decoder to decode to text
test_predictions_decoded = tf_keras_backend.get_value(tf_keras_backend.ctc_decode(test_predictions_encoded,
                                                                                  input_length = np.ones(test_predictions_encoded.shape[0])*test_predictions_encoded.shape[1],
                                                                                  greedy=True)[0][0])
test_predictions_decoded.shape

(1, 32)

In [ ]:
def numbered_array_to_text(numbered_array):
    numbered_array = numbered_array[numbered_array != -1]
    return "".join(letters[i] for i in numbered_array)

letters = [' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
           '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?',
           'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
           'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
           'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

num_classes = len(letters) + 1

#predicting the 5 test data labels
for i in range(1):
  #print("original_text = ", original_test_texts[i])
  print("predicted text = ", numbered_array_to_text(test_predictions_decoded[i]))
  print()

predicted text =  7MEB3Z4



In [ ]:
#autocorrecting the pedictions
test_sentences = []
for i in range(1):
    string = numbered_array_to_text(test_predictions_decoded[i])
    #remove characters
    k = ''.join(e for e in string if e.isalnum())
    #remove numbers
    k = ''.join([i for i in k if not i.isdigit()])
    test_sentences.append(k)

test_sentences

['MEBZ']

#### **************** Model 2 ********************

In [ ]:
#importing the libraries
import os, sys
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [ ]:
#Redifining input and output sequences
input_sentences = []
output_sentences = []
output_sentences_inputs = []

count = 0
for line in open(r'/content/drive/My Drive/AI in Manufac/project/Translation/data/fra.txt', encoding="utf-8"):
    count += 1
    #print(line.rstrip().split('\t')[0])

    if count > NUM_SENTENCES:
        break

    if '\t' not in line:
        continue
    sss = line.rstrip().split('\t')
    input_sentence = sss[0]
    output = sss[1]
    #print(input_sentence, output)
    #sss = line.rstrip().split('\t')
    '''
    for at, bt in enumerate(sss):
      if at==0:
        input_sentence = bt
      elif at==1:
        output = bt
        break
    '''
    output_sentence = output + ' <eos>'
    output_sentence_input = '<sos> ' + output

    input_sentences.append(input_sentence)
    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)

#print("num samples input:", len(input_sentences))
#print("num samples output:", len(output_sentences))
#print("num samples output input:", len(output_sentences_inputs))
#for i in range(4):
#  print("see:", input_sentences[i], output_sentences[i], output_sentences_inputs[i])

In [ ]:
#demo
print(input_sentences[172])
print(output_sentences[172])
print(output_sentences_inputs[172])

I'm Tom.
Je suis Tom. <eos>
<sos> Je suis Tom.


In [ ]:
#Tokenization splitting sentences into words
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the input: 3518
Length of longest sentence in input: 6
Total unique words in the output: 9546
Length of longest sentence in the output: 12


In [ ]:
print(word2idx_inputs["i'm"])
print(word2idx_inputs["ill"])

6
539


In [ ]:
print(word2idx_outputs["<sos>"])
print(word2idx_outputs["je"])
print(word2idx_outputs["suis"])
print(word2idx_outputs["malade."])

2
3
6
184


In [ ]:
#encoding and decoding 
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

encoder_input_sequences.shape: (20000, 6)
encoder_input_sequences[172]: [0 0 0 0 6 7]
decoder_input_sequences.shape: (20000, 12)
decoder_input_sequences[172]: [ 2  3  6 52  0  0  0  0  0  0  0  0]


In [ ]:
#word embeddings
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open(r'/content/drive/My Drive/AI in Manufac/project/Translation/data/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [ ]:
#creating a embedding matrix 
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
print(embeddings_dictionary["ill"])

[ 0.12648    0.1366     0.22192   -0.025204  -0.7197     0.66147
  0.48509    0.057223   0.13829   -0.26375   -0.23647    0.74349
  0.46737   -0.462      0.20031   -0.26302    0.093948  -0.61756
 -0.28213    0.1353     0.28213    0.21813    0.16418    0.22547
 -0.98945    0.29624   -0.62476   -0.29535    0.21534    0.92274
  0.38388    0.55744   -0.14628   -0.15674   -0.51941    0.25629
 -0.0079678  0.12998   -0.029192   0.20868   -0.55127    0.075353
  0.44746   -0.71046    0.75562    0.010378   0.095229   0.16673
  0.22073   -0.46562   -0.10199   -0.80386    0.45162    0.45183
  0.19869   -1.6571     0.7584    -0.40298    0.82426   -0.386
  0.0039546  0.61318    0.02701   -0.3308    -0.095652  -0.082164
  0.7858     0.13394   -0.32715   -0.31371   -0.20247   -0.73001
 -0.49343    0.56445    0.61038    0.36777   -0.070182   0.44859
 -0.61774   -0.18849    0.65592    0.44797   -0.10469    0.62512
 -1.9474    -0.60622    0.073874   0.50013   -1.1278    -0.42066
 -0.37322   -0.50538    0

In [ ]:
print(embedding_matrix[539])

[ 0.12648     0.1366      0.22192    -0.025204   -0.71969998  0.66147
  0.48508999  0.057223    0.13829    -0.26374999 -0.23647     0.74348998
  0.46737    -0.46200001  0.20031001 -0.26302001  0.093948   -0.61756003
 -0.28213     0.1353      0.28213     0.21813001  0.16418     0.22547001
 -0.98944998  0.29624    -0.62475997 -0.29534999  0.21534     0.92273998
  0.38387999  0.55743998 -0.14628001 -0.15673999 -0.51941001  0.25628999
 -0.0079678   0.12998    -0.029192    0.20868    -0.55127001  0.075353
  0.44746    -0.71046001  0.75562     0.010378    0.095229    0.16673
  0.22073001 -0.46562001 -0.10199    -0.80386001  0.45162001  0.45183
  0.19869    -1.65709996  0.75840002 -0.40298     0.82426    -0.38600001
  0.0039546   0.61317998  0.02701    -0.3308     -0.095652   -0.082164
  0.78579998  0.13394    -0.32714999 -0.31371    -0.20247    -0.73000997
 -0.49342999  0.56445003  0.61037999  0.36776999 -0.070182    0.44859001
 -0.61773998 -0.18849     0.65592003  0.44797    -0.10469     0.

In [ ]:
embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len)

In [ ]:
# Creating Model
#performed on hot encoding and intialized encoder and decoder
decoder_targets_one_hot = np.zeros((
        len(input_sentences),
        max_out_len,
        num_words_output
    ),
    dtype='float32'
)
print(decoder_targets_one_hot.shape)

(20000, 12, 9547)


In [ ]:
#creating the output decoder for the output
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
for i, d in enumerate(decoder_output_sequences):
#for i, d in enumerate(decoder_input_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [ ]:
#creating an encoder for input
encoder_inputs_placeholder = Input(shape=(max_input_len,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [ ]:
#creating an input decoder
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)

In [ ]:
#dense layer with soft max activation function 
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
#compiling the model
model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
#training the model
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1,
)

Epoch 1/20
282/282 [==============================] - 17s 62ms/step - loss: 2.1584 - accuracy: 0.6989 - val_loss: 2.2110 - val_accuracy: 0.6842
Epoch 2/20
282/282 [==============================] - 17s 59ms/step - loss: 1.5840 - accuracy: 0.7629 - val_loss: 1.9784 - val_accuracy: 0.7153
Epoch 3/20
282/282 [==============================] - 17s 59ms/step - loss: 1.3698 - accuracy: 0.7943 - val_loss: 1.8731 - val_accuracy: 0.7330
Epoch 4/20
282/282 [==============================] - 16s 58ms/step - loss: 1.2401 - accuracy: 0.8102 - val_loss: 1.7802 - val_accuracy: 0.7419
Epoch 5/20
282/282 [==============================] - 17s 59ms/step - loss: 1.1412 - accuracy: 0.8223 - val_loss: 1.7227 - val_accuracy: 0.7500
Epoch 6/20
282/282 [==============================] - 16s 59ms/step - loss: 1.0645 - accuracy: 0.8316 - val_loss: 1.6876 - val_accuracy: 0.7566
Epoch 7/20
282/282 [==============================] - 17s 59ms/step - loss: 1.0007 - accuracy: 0.8400 - val_loss: 1.6764 - val_accuracy:

In [ ]:
#Modifying the Model
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [ ]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [ ]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [ ]:
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

In [ ]:
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

### making predictions

In [ ]:
#Making Predictions
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [ ]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)

In [ ]:
test_sentences = ['EDUCATION']
test_sentences

['EDUCATION']

In [ ]:
#tokenising the test data
test_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
test_tokenizer.fit_on_texts(test_sentences)
test_integer_seq = test_tokenizer.texts_to_sequences(test_sentences)

max_test_len = max(len(sen) for sen in test_integer_seq)

encoder_test_sequences = pad_sequences(test_integer_seq, maxlen=max_test_len)
encoder_test_sequences

array([[1]], dtype=int32)

In [ ]:
for i in range(1):
  #i = np.random.choice(len(test_sentences))
  #print(i)
  input_seq = encoder_test_sequences[i:i+1]
  print(input_seq)
  translation = translate_sentence(input_seq)
  print('Input:', test_sentences[i])
  print('Response:', translation)
  print('--------------')

[[1]]
Input: EDUCATION
Response: je m'en suis !
--------------
